Преподаватель: Артур Сапрыкин
Использовать dataset MovieLens (https://grouplens.org/datasets/movielens/latest/)

Построить рекомендации (регрессия, предсказываем оценку) на фичах:
TF-IDF на тегах и жанрах
Средние оценки (+ median, variance, etc.) пользователя и фильма
Оценить RMSE на тестовой выборке

NOTES from Slack: 
В первом ноубуке мы высчитывали средние, медианы, а во втором векторизировали теги и жанры. Так вот, твоя задача состоит в том, чтобы 

!!!предсказать оценку, которую поставит пользователь. #userid-movieid-rating

Можно добавить следующие признаки:
-средняя оценка фильма по всем пользователям (гипотетически, у тебя уже есть статистика по фильму до того, как ты даёшь прогноз)
-средняя оценка, которую ставит пользователь.
-среднее количество тегов фильма
-добавить tf-idf по жанрам.
Это пример тех признаков, которые ты можешь сформировать. А итоговая метка - это рейтинг. В датасета информация об этом есть.

#Карасев Вадим
Подскажите по домашке по рекомендательным системам.
Там написано
1.Построить рекомендации (регрессия, предсказываем оценку)
на фичах:
■TF-IDF на тегах и жанрах
я не понял, сразу на 2х фичах? На лекции мы вроде делали последовательно, сначала на одной, потом на другой и сравнивали.

Ааа  кажется понял. Сделать то же самое но предсказание другой моделью.
Всем спасибо!)

Регрессию нужно вставлять в предсказание оценки. Там же рейтинги от 0 до 5. Это и будет таргет

@Артур Сапрыкин то есть мы должны сформировать новый дата сет с например указанными фичами?
Но я запутался с tf-idf. В резултате мы получаем матрицу весов или втречаемости того или иного жанра/тега и т.п. Но там размерность другая и я не понимаю как ее связать с исх данными(

In [1]:
# Использовать dataset MovieLens

# download data
import numpy as np
import pandas as pd

links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

# 'ratings' is Pivot table. Here I will add new TFiDF features from 'tags' and 'movies'
# Here I will create Regression model
ratings.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [2]:
ratings.shape

(100836, 4)

In [3]:
# Средние оценки (+ median, variance, etc.) пользователя и фильма
# create dict with avarage values
user_aver_rating_dict = ratings[["userId", "rating"]].groupby(['userId']).mean().to_dict()
movie_aver_rating_dict = ratings[["movieId", "rating"]].groupby(['movieId']).mean().to_dict()
user_rat_numb_dict = ratings[["userId", "rating"]].groupby(['userId']).count().to_dict()
user_rat_numb_dict
# print(age_dict.get('Age').get('Don'))

{'rating': {1: 232,
  2: 29,
  3: 39,
  4: 216,
  5: 44,
  6: 314,
  7: 152,
  8: 47,
  9: 46,
  10: 140,
  11: 64,
  12: 32,
  13: 31,
  14: 48,
  15: 135,
  16: 98,
  17: 105,
  18: 502,
  19: 703,
  20: 242,
  21: 443,
  22: 119,
  23: 121,
  24: 110,
  25: 26,
  26: 21,
  27: 135,
  28: 570,
  29: 81,
  30: 34,
  31: 50,
  32: 102,
  33: 156,
  34: 86,
  35: 23,
  36: 60,
  37: 21,
  38: 78,
  39: 100,
  40: 103,
  41: 217,
  42: 440,
  43: 114,
  44: 48,
  45: 399,
  46: 42,
  47: 140,
  48: 33,
  49: 21,
  50: 310,
  51: 359,
  52: 130,
  53: 20,
  54: 33,
  55: 25,
  56: 46,
  57: 476,
  58: 112,
  59: 107,
  60: 22,
  61: 39,
  62: 366,
  63: 271,
  64: 517,
  65: 34,
  66: 345,
  67: 36,
  68: 1260,
  69: 46,
  70: 62,
  71: 35,
  72: 45,
  73: 210,
  74: 177,
  75: 69,
  76: 119,
  77: 29,
  78: 61,
  79: 64,
  80: 167,
  81: 26,
  82: 227,
  83: 118,
  84: 293,
  85: 34,
  86: 70,
  87: 21,
  88: 56,
  89: 518,
  90: 54,
  91: 575,
  92: 24,
  93: 97,
  94: 56,
  95: 168,
  

In [4]:
# map dict to ratings
ratings['user_aver_rating'] = ratings['userId'].map(user_aver_rating_dict.get('rating'))
ratings['movie_aver_rating'] = ratings['movieId'].map(movie_aver_rating_dict.get('rating'))
ratings['user_rat_numb'] = ratings['userId'].map(user_rat_numb_dict.get('rating'))

In [5]:
ratings.tail(5)

,userId,movieId,rating,timestamp,user_aver_rating,movie_aver_rating,user_rat_numb
100831,610,166534,4.0,1493848402,3.688556,3.333333,1302
100832,610,168248,5.0,1493850091,3.688556,4.142857,1302
100833,610,168250,5.0,1494273047,3.688556,3.633333,1302
100834,610,168252,5.0,1493846352,3.688556,4.280000,1302
100835,610,170875,3.0,1493846415,3.688556,2.333333,1302


In [6]:
# copy
ratings0 = ratings.copy()
ratings0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
userId               100836 non-null int64
movieId              100836 non-null int64
rating               100836 non-null float64
timestamp            100836 non-null int64
user_aver_rating     100836 non-null float64
movie_aver_rating    100836 non-null float64
user_rat_numb        100836 non-null int64
dtypes: float64(3), int64(4)
memory usage: 5.4 MB


In [7]:
# Run basic model. With this model I will estimate additional value from tfIdf
X = ratings0.loc[:, ratings0.columns !='rating']
y = ratings0['rating']

# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=42)

In [8]:
# Оценить RMSE на тестовой выборке

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
import math

reg = RandomForestRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

from sklearn.metrics import mean_squared_error
print(f'RMSE: {math.sqrt(mean_squared_error(y_test, y_pred))}')

# NOTE: basic model without tfidf

RMSE: 0.801863782079543


# Add tfIdf

## Working with movies

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
def change_string(row):
    s = row['genres']
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [11]:
movies0 = movies.copy()

In [12]:
# tokenization
movies0['genres_list'] = movies0.apply(change_string, axis = 1)
movies0.head()

,movieId,title,genres,genres_list
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy,Comedy


In [13]:
movies0.shape

(9742, 4)

In [14]:
# vocabulary building
bards_words = movies0['genres_list']
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

print("Размер словаря: {}".format(len(vect.vocabulary_)))
print("Содержимое словаря:\n {}".format(vect.vocabulary_))

Размер словаря: 20
Содержимое словаря:
 {'adventure': 1, 'animation': 2, 'children': 3, 'comedy': 4, 'fantasy': 8, 'romance': 15, 'drama': 7, 'action': 0, 'crime': 5, 'thriller': 17, 'horror': 10, 'mystery': 13, 'scifi': 16, 'war': 18, 'musical': 12, 'documentary': 6, 'imax': 11, 'western': 19, 'filmnoir': 9, 'nogenreslisted': 14}


In [15]:
# sparse matrix encoding
bag_of_words_tr = vect.transform(bards_words)
print("bag_of_words: {}".format(repr(bag_of_words_tr)))

bag_of_words: <9742x20 sparse matrix of type '<class 'numpy.int64'>'
	with 22084 stored elements in Compressed Sparse Row format>


In [16]:
print("Плотное представление bag_of_words:\n{}".format(
 bag_of_words_tr.toarray()))

Плотное представление bag_of_words:
[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [17]:
# TFiDF scaling
# term frequency-inverse document frequency, tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import make_pipeline
# TfidfVectorizer()
tfidf_transformer_movies = TfidfTransformer()
bag_of_words_tfidf = tfidf_transformer_movies.fit_transform(bag_of_words_tr)
bag_of_words_tfidf
bag_of_words_tfidf.toarray()


array([[0.        , 0.41684567, 0.51622547, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.51236121, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57860574, 0.        , 0.81560738, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [18]:
# Shorter execution TFiDF scaling
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
tfidf_v = TfidfVectorizer().fit(bards_words)

bag_of_words_tfidf_simple = tfidf_v.transform(bards_words)
# X_train = vectorizer.transform(text_train)
bag_of_words_tfidf_simple.toarray()


array([[0.        , 0.41684567, 0.51622547, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.51236121, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57860574, 0.        , 0.81560738, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
# get features names to create dataframe
feature_names = np.array(tfidf_v.get_feature_names())
feature_names

array(['action', 'adventure', 'animation', 'children', 'comedy', 'crime',
       'documentary', 'drama', 'fantasy', 'filmnoir', 'horror', 'imax',
       'musical', 'mystery', 'nogenreslisted', 'romance', 'scifi',
       'thriller', 'war', 'western'], dtype='<U14')

In [20]:
bag_of_words_tfidf_simple.toarray().shape

(9742, 20)

In [21]:
# look to size - it's the same than number of rows as in 'movies'
bag_of_words_tfidf.toarray().shape

(9742, 20)

Result the same, let's use simple way

In [22]:
movies_tfidf = pd.DataFrame(bag_of_words_tfidf.toarray(),columns = feature_names)
movies_tfidf.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [23]:
# create DataFrame bag_of_words_tfidf_simple
movies_tfidf_simple = pd.DataFrame(bag_of_words_tfidf_simple.toarray(),columns = feature_names)
movies_tfidf_simple.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [24]:
# movies with tfidf features
movies_tfidf = movies0.join(movies_tfidf_simple)

# result = pd.merge(movies, movies_tfidf_simple, how='inner', 
#                  left_on=['userId', 'movieId', 'timestamp'],
#                  right_on=['userId', 'movieId', 'timestamp'],
#                   suffixes=('', '_y')
#                  )
# result.tail(30)

movies_tfidf.head()

,movieId,title,genres,genres_list,action,adventure,animation,children,comedy,crime,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy,0.0,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure Children Fantasy,0.0,0.512361,0.000000,0.620525,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy Romance,0.0,0.000000,0.000000,0.000000,0.570915,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy Drama Romance,0.0,0.000000,0.000000,0.000000,0.505015,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,Comedy,0.0,0.000000,0.000000,0.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [25]:
movies_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 24 columns):
movieId           9742 non-null int64
title             9742 non-null object
genres            9742 non-null object
genres_list       9742 non-null object
action            9742 non-null float64
adventure         9742 non-null float64
animation         9742 non-null float64
children          9742 non-null float64
comedy            9742 non-null float64
crime             9742 non-null float64
documentary       9742 non-null float64
drama             9742 non-null float64
fantasy           9742 non-null float64
filmnoir          9742 non-null float64
horror            9742 non-null float64
imax              9742 non-null float64
musical           9742 non-null float64
mystery           9742 non-null float64
nogenreslisted    9742 non-null float64
romance           9742 non-null float64
scifi             9742 non-null float64
thriller          9742 non-null float64
war               

In [26]:
# drop colunms with Object data to fit model
movies_tfidf.drop(['title','genres', 'genres_list' ], axis=1, inplace = True)

In [27]:
# join ratings with movies_tfidf
ratings_movies = ratings0.set_index('movieId').join(movies_tfidf.set_index('movieId'), how='left')
ratings_movies.head()

,userId,rating,timestamp,user_aver_rating,movie_aver_rating,user_rat_numb,action,adventure,animation,children,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,4.0,964982703,4.366379,3.92093,232,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,4.0,847434962,3.636364,3.92093,44,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,4.5,1106635946,3.230263,3.92093,152,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,2.5,1510577970,3.448148,3.92093,135,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,4.5,1305696483,4.209524,3.92093,105,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
ratings_movies.shape

(100836, 26)

In [29]:
ratings0.shape

(100836, 7)

# Create model and check added value from tfidf(movies)

In [30]:
# Run basic model. With this model I will estimate additional value from tfIdf
X = ratings_movies.loc[:, ratings_movies.columns !='rating']
y = ratings_movies['rating']

# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=42)


# Оценить RMSE на тестовой выборке

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet

reg = RandomForestRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

from sklearn.metrics import mean_squared_error
import math
print(f'RMSE: {math.sqrt(mean_squared_error(y_test, y_pred))}')

# NOTE: basic model better when with tfidf

RMSE: 0.7977354487674871


# Conclusion: Basic model better when with tfidf

# Add tags the same way

In [88]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,boxing story,1445715207
4,2,89774,mma,1445715200


In [89]:
# run lowercase for 'tag'
tags['tag'] = tags['tag'].str.lower()
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,boxing story,1445715207
4,2,89774,mma,1445715200


In [90]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
userId       3683 non-null int64
movieId      3683 non-null int64
tag          3683 non-null object
timestamp    3683 non-null int64
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [91]:
# create backup
tags0 = tags.copy()

# drop duplicates if user write comments several times. Keep 'first' comment (before correspondence or discussion)
tags0.drop_duplicates(subset=['movieId', 'userId'], keep="first", inplace = True)

In [92]:
# NOTE: look to reduce of rows. It means same users write several tags for 1 movies.
# If I just drop_duplicates I lose information. I should summirise tags for userId_movieId
tags0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1775 entries, 0 to 3682
Data columns (total 4 columns):
userId       1775 non-null int64
movieId      1775 non-null int64
tag          1775 non-null object
timestamp    1775 non-null int64
dtypes: int64(3), object(1)
memory usage: 69.3+ KB


In [93]:
# create backup
tags1 = tags.copy()

# create new column
tags1["userId_movieId"] = tags1["userId"].astype(str) +'_'+ tags1["movieId"].astype(str)
tags1.head()

,userId,movieId,tag,timestamp,userId_movieId
0,2,60756,funny,1445714994,2_60756
1,2,60756,highly quotable,1445714996,2_60756
2,2,60756,will ferrell,1445714992,2_60756
3,2,89774,boxing story,1445715207,2_89774
4,2,89774,mma,1445715200,2_89774


In [94]:
from tqdm import tqdm_notebook

tag_strings = []
userId_movieId_s = []

for userId_movieId, group in tqdm_notebook(tags1.groupby('userId_movieId')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    userId_movieId_s.append(userId_movieId)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [95]:
# check
print(len(userId_movieId_s), len(tag_strings))
print(userId_movieId_s[:1], tag_strings[:1] )

tags_movies_unique = pd.DataFrame(list(zip(tag_strings, userId_movieId_s)), 
                                              columns =['unique_tag', 'userId_movieId_s'])

tags_movies_unique.head()


1775 1775
['103_260'] ['epic greatsoundtrack']


,unique_tag,userId_movieId_s
0,epic greatsoundtrack,103_260
1,gooddialogue greatsoundtrack nonlinear,103_296
2,adventure,106_106489
3,everythingyouwantishere,106_4896
4,classicscifi engrossingadventure epic,112_260


In [96]:
tags_movies_unique[tags_movies_unique['userId_movieId_s'] =='2_60756']

,unique_tag,userId_movieId_s
71,funny highlyquotable willferrell,2_60756


In [97]:
tags[(tags['userId'] == 2) & (tags['movieId']==60756)]
# NOTE: Ckeck operation - Ok

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,highly quotable,1445714996
2,2,60756,will ferrell,1445714992


In [98]:
type(tags_movies_unique)

pandas.core.frame.DataFrame

In [99]:
# split columns 
# tags_movies_unique[['userId', 'movieId']] = tags_movies_unique['userId_movieId_s'].str.split("_",expand=True,)


In [100]:
# tags_movies_unique.drop(['userId_movieId_s'], axis=1, inplace = True)
tags_movies_unique.head()

,unique_tag,userId_movieId_s
0,epic greatsoundtrack,103_260
1,gooddialogue greatsoundtrack nonlinear,103_296
2,adventure,106_106489
3,everythingyouwantishere,106_4896
4,classicscifi engrossingadventure epic,112_260


In [101]:
tags_movies_unique.shape

(1775, 2)

In [102]:
tags_movies_unique.drop_duplicates(inplace = True)

In [103]:
tags_movies_unique.shape

(1775, 2)

In [104]:
# vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
tfidf_v_tag = TfidfVectorizer(stop_words = 'english').fit(tags_movies_unique.unique_tag)

bag_of_words_tfidf_tag = tfidf_v_tag.transform(tags_movies_unique.unique_tag)
# X_train = vectorizer.transform(text_train)
bag_of_words_tfidf_tag.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [105]:
bag_of_words_tfidf_tag
# much more columns than in movies

<1775x1471 sparse matrix of type '<class 'numpy.float64'>'
	with 3707 stored elements in Compressed Sparse Row format>

In [106]:
# get features names to create dataframe
feature_names_tags = np.array(tfidf_v_tag.get_feature_names())
feature_names_tags[:20]

array(['06oscarnominatedbestmovieanimation', '1900s', '1920s', '1950s',
       '1960s', '1970s', '1980s', '1990s', '2001like', '2danimation',
       '70mm', '80', 'aardman', 'abortion', 'absorbing', 'abstract',
       'academyaward', 'accident', 'achronological', 'acleverchefrat'],
      dtype='<U54')

In [107]:
feature_names_tags.shape

(1471,)

In [108]:
# join tags with bag_of_words_tfidf_tag
tags_tfidf = pd.DataFrame(bag_of_words_tfidf_tag.toarray(), columns = feature_names_tags)
tags_tfidf.shape

(1775, 1471)

In [109]:
len(userId_movieId_s)

1775

In [145]:
# add 'userId_movieId_s' to tags_tfidf
tags_tfidf['userId_movieId'] = userId_movieId_s
tags_tfidf

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,userId_movieId
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103_260
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103_296
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106_106489
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106_4896
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112_260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62_96861
1771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62_99114
1772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63_260
1773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76_260


In [146]:
# check the data id Ok
tags_tfidf[tags_tfidf['1950s']>0]

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,userId_movieId
251,0.0,0.0,0.0,0.776211,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474_1103
604,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474_2664


In [150]:
# ratings_movies = ratings_movies.reset_index()
ratings_movies["userId_movieId"] = ratings_movies["userId"].astype(str) +'_'+ ratings_movies["movieId"].astype(str)
# ratings_movies.set_index('userId_movieId')
ratings_movies.head()

,movieId,userId,rating,timestamp,user_aver_rating,movie_aver_rating,user_rat_numb,action,adventure,animation,...,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,userId_movieId
0,1,1,4.0,964982703,4.366379,3.92093,232,0.0,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_1
1,1,5,4.0,847434962,3.636364,3.92093,44,0.0,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5_1
2,1,7,4.5,1106635946,3.230263,3.92093,152,0.0,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7_1
3,1,15,2.5,1510577970,3.448148,3.92093,135,0.0,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15_1
4,1,17,4.5,1305696483,4.209524,3.92093,105,0.0,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17_1


In [152]:
tags_tfidf[tags_tfidf['userId_movieId']=='2_60756'].T

,71
06oscarnominatedbestmovieanimation,0
1900s,0
1920s,0
1950s,0
1960s,0
...,...
zither,0
zoekazan,0
zombies,0
zooeydeschanel,0


In [153]:
ratings_movies[ratings_movies['userId_movieId']=='2_60756']

,movieId,userId,rating,timestamp,user_aver_rating,movie_aver_rating,user_rat_numb,action,adventure,animation,...,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,userId_movieId
87775,60756,2,5.0,1445714980,3.948276,3.553571,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2_60756


In [165]:
tags_tfidf

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,userId_movieId
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103_260
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103_296
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106_106489
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106_4896
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112_260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62_96861
1771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62_99114
1772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63_260
1773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76_260


In [170]:
tags_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Columns: 1472 entries, 06oscarnominatedbestmovieanimation to userId_movieId
dtypes: float64(1471), object(1)
memory usage: 19.9+ MB


In [166]:
# join ratings with movies_tfidf
result = pd.merge(ratings_movies, tags_tfidf, how = 'left', on = 'userId_movieId')
result[result['userId_movieId']=='2_60756']

,movieId,userId,rating,timestamp,user_aver_rating,movie_aver_rating,user_rat_numb,action_x,adventure_x,animation_x,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
87775,60756,2,5.0,1445714980,3.948276,3.553571,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [169]:
tags_tfidf[tags_tfidf['userId_movieId']=='2_60756']

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,userId_movieId
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2_60756


### NOTE: small amount of tags (userId_movieId) merge with ratings_movies. It meance that people usually don't write tags

In [171]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Columns: 1499 entries, movieId to zooeydeschanel
dtypes: float64(1494), int64(4), object(1)
memory usage: 1.1+ GB


In [172]:
result.shape

# bag shape. Size not the same

(100836, 1499)

In [175]:
# fillna for evoid rising Error with model
result = result.fillna(0)

In [177]:
result

,movieId,userId,rating,timestamp,user_aver_rating,movie_aver_rating,user_rat_numb,action_x,adventure_x,animation_x,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,1,4.0,964982703,4.366379,3.92093,232,0.000000,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5,4.0,847434962,3.636364,3.92093,44,0.000000,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,7,4.5,1106635946,3.230263,3.92093,152,0.000000,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,15,2.5,1510577970,3.448148,3.92093,135,0.000000,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,17,4.5,1305696483,4.209524,3.92093,105,0.000000,0.416846,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,193581,184,4.0,1537109082,3.705224,4.00000,134,0.436010,0.000000,0.614603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100832,193583,184,3.5,1537109545,3.705224,3.50000,134,0.000000,0.000000,0.682937,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100833,193585,184,3.5,1537109805,3.705224,3.50000,134,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100834,193587,184,3.5,1537110021,3.705224,3.50000,134,0.578606,0.000000,0.815607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Create model and check added value from tfidf(tags)

In [179]:
# Run model with addition information about movies genres and users tags.
X = result.loc[:, result.columns !='rating']
y = result['rating']

# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=42)


# Оценить RMSE на тестовой выборке

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet

reg = RandomForestRegressor(n_jobs = -1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
import math

from sklearn.metrics import mean_squared_error
print(f'RMSE: {math.sqrt(mean_squared_error(y_test, y_pred))}')

# NOTE: 
# basic model RMSE: 0.801863782079543
# basic + tfidf movies genres RMSE: 0.7977354487674871
# basic + tfidf movies genres + tfidf users tags RMSE: 0.7895352738599626

# Conclusion: in this home work adding tfIdf don't give addition accuracy

RMSE: 0.7895352738599626
